## Decoding Strategies
Author: Victoria Pedlar

This notebook explores open-ended text generation for isiZulu.

### Transformer Language Model

In [392]:
import os

# Get the current working directory
cwd = os.getcwd()
print(cwd)

# Change the current working directory
os.chdir('/Users/victoriapedlar/repos/isizulu-text-generation/')

# Get the current working directory again
cwd = os.getcwd()
print(cwd)

/Users/victoriapedlar/repos/isizulu-text-generation
/Users/victoriapedlar/repos/isizulu-text-generation


In [393]:
# %pip install -r transformer_requirements.txt
# %cd src/transformers
# %pip install .
# %cd /Users/victoriapedlar/repos/isizulu-text-generation/

In [394]:
import torch
import nltk
import transformers
from transformers import GPT2TokenizerFast
import sys
sys.path.insert(0,'/Users/victoriapedlar/repos/isizulu-text-generation/scripts')
import layer_switching_gpt2
from layer_switching_gpt2 import LayerSwitchingGPT2Config, GPT2LayerSwitchingLMHeadModel

In [395]:
tokenizer = GPT2TokenizerFast.from_pretrained('experiments/trained_models/transformer/tokenizers')

In [396]:
# Load the model configuration
config = LayerSwitchingGPT2Config.from_pretrained("experiments/trained_models/transformer/config.json")
# Load the model from the checkpoint
model = GPT2LayerSwitchingLMHeadModel.from_pretrained("experiments/trained_models/transformer", config=config)

In [397]:
# Set the tokenizer attribute of the model
model.tokenizer = tokenizer

In [398]:
# Define a list of decoding strategies to try
decoding_strategies = ['argmax', 'beam_search', 'nucleus_sampling', 'top_k_sampling', 'temperature_sampling', 'entmax']

There are several decoding strategies available for open-ended language generation using transformer models:

1. Argmax decoding: This is a type of greedy search, where the model chooses the most likely next word at each step.

2. Beam search: In beam search, the model considers a fixed number of top options at each step, and expands the search space by adding the next most likely words to these options. This can produce more coherent and diverse output than argmax decoding, but can also be slower.

3. Nucleus sampling: Nucleus sampling is a variant of top-k sampling, where the model samples from the set of most likely words, with probability proportional to their likelihood. This can produce more diverse output than argmax decoding, but may still be biased towards the most likely words.

4. Top-k sampling: In top-k sampling, the model samples from the top-k most likely words at each step. This can produce more diverse output than argmax decoding, but may still be biased towards the most likely words.

5. Temperature sampling: In temperature sampling, the model samples from its output distribution with a specified temperature. A high temperature will produce more diverse output, but may also introduce more errors and randomness. A low temperature will produce less diverse output, but may be more accurate.

6. Entmax sampling

In [399]:
def extract_prompts_and_references(file_path):
    prompts = []
    references = []

    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            prompt = ' '.join(line.split()[:5])
            reference = line[len(prompt)+1:]

            prompts.append(prompt)
            references.append(reference)

    return prompts, references

In [400]:
prompts, references = extract_prompts_and_references("data/test/isolezwe.txt")

In [401]:
# Calculate the average number of tokens in each reference
total_tokens = sum(len(tokenizer.encode(ref)) for ref in references)
average_tokens = total_tokens / len(references)
max_length = round(average_tokens) + 2

print(f"The average number of tokens in the references is {average_tokens:.2f} tokens.")

The average number of tokens in the references is 29.14 tokens.


In [402]:
def generate_text(prompt, strategy, hyperparameters, max_length=max_length):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    prompt_length = input_ids.shape[1]

    if strategy == 'argmax':
        # Generate text with argmax decoding
        num_beams = hyperparameters.get('num_beams', 1)
        output_ids = model.generate(input_ids, max_length=max_length+prompt_length, num_beams=num_beams, do_sample=False)
    else:
        # Generate text with the specified decoding strategy and hyperparameters
        if strategy == 'beam_search':
            beam_size = hyperparameters.get('beam_size', 1)
            no_repeat_ngram_size = hyperparameters.get('no_repeat_ngram_size', 2)
            output_ids = model.generate(input_ids, num_beams=beam_size, no_repeat_ngram_size=no_repeat_ngram_size, max_length=max_length+prompt_length)
        elif strategy == 'nucleus_sampling':
            top_p = hyperparameters.get('top_p', 0.9)
            output_ids = model.generate(input_ids, do_sample=True, max_length=max_length+prompt_length, top_p=top_p)
        elif strategy == 'top_k_sampling':
            k = hyperparameters.get('k', 10)
            output_ids = model.generate(input_ids, do_sample=True, max_length=max_length+prompt_length, top_k=k)
        elif strategy == 'temperature_sampling':
            temperature = hyperparameters.get('temperature', 1.0)
            output_ids = model.generate(input_ids, do_sample=True, max_length=max_length+prompt_length, temperature=temperature)
        elif strategy == 'entmax_sampling':
            alpha = hyperparameters.get('alpha', 1.5)
            output_ids = model.generate(input_ids, do_sample=True, max_length=max_length+prompt_length, entmax=True, alpha=alpha)

    # Remove the prompt tokens from the generated text
    if strategy == 'argmax':
        generated_text = model.tokenizer.decode(output_ids[0][prompt_length:].tolist(), skip_special_tokens=True)
    else:
        generated_text = model.tokenizer.decode(output_ids[0][prompt_length:-1].tolist(), skip_special_tokens=True)

    generated_text = generated_text.replace('\n', '')
    
    return generated_text

In [403]:
from itertools import chain

def pad_sequence(sequence, n, pad_left=False, pad_right=False,
                 left_pad_symbol=None, right_pad_symbol=None):
    """
    Returns a padded sequence of items before ngram extraction.
    :param sequence: the source data to be padded
    :type sequence: sequence or iter
    :param n: the degree of the ngrams
    :type n: int
    :param pad_left: whether the ngrams should be left-padded
    :type pad_left: bool
    :param pad_right: whether the ngrams should be right-padded
    :type pad_right: bool
    :param left_pad_symbol: the symbol to use for left padding (default is None)
    :type left_pad_symbol: any
    :param right_pad_symbol: the symbol to use for right padding (default is None)
    :type right_pad_symbol: any
    :rtype: sequence or iter
    """
    sequence = iter(sequence)
    if pad_left:
        sequence = chain((left_pad_symbol,) * (n - 1), sequence)
    if pad_right:
        sequence = chain(sequence, (right_pad_symbol,) * (n - 1))
    return sequence


def ngrams(sequence, n, pad_left=False, pad_right=False,
           left_pad_symbol=None, right_pad_symbol=None):
    """
    Return the ngrams generated from a sequence of items, as an iterator.
    :param sequence: the source data to be converted into ngrams
    :type sequence: sequence or iter
    :param n: the degree of the ngrams
    :type n: int
    :param pad_left: whether the ngrams should be left-padded
    :type pad_left: bool
    :param pad_right: whether the ngrams should be right-padded
    :type pad_right: bool
    :param left_pad_symbol: the symbol to use for left padding (default is None)
    :type left_pad_symbol: any
    :param right_pad_symbol: the symbol to use for right padding (default is None)
    :type right_pad_symbol: any
    :rtype: sequence or iter
    """
    sequence = pad_sequence(sequence, n, pad_left, pad_right,
                            left_pad_symbol, right_pad_symbol)

    history = []
    while n > 1:
        history.append(next(sequence))
        n -= 1
    for item in sequence:
        history.append(item)
        yield tuple(history)
        del history[0]

def distinct_n_sentence_level(sentence, n):
    """
    Compute distinct-N for a single sentence.
    :param sentence: a list of words.
    :param n: int, ngram.
    :return: float, the metric value.
    """
    if len(sentence) == 0:
        return 0.0  # Prevent a zero division
    distinct_ngrams = set(ngrams(sentence, n))
    return len(distinct_ngrams) / len(sentence)


def distinct_n_corpus_level(sentences, n):
    """
    Compute average distinct-N of a list of sentences (the corpus).
    :param sentences: a list of sentence.
    :param n: int, ngram.
    :return: float, the average value.
    """
    return sum(distinct_n_sentence_level(sentence, n) for sentence in sentences) / len(sentences)

In [419]:
from sacrebleu.metrics import BLEU, CHRF
# from cer import calculate_cer_corpus
from evaluate import load

def evaluate(generated_texts, reference_texts):    
    
    # # Compute perplexity
    # Concatenate all generated texts using newline characters
    # concatenated_generated_text = '\n'.join(generated_texts)
    # input_ids = tokenizer.encode(concatenated_generated_text, return_tensors="pt")
    
    # # Divide input_ids into smaller chunks if the length exceeds the model's max length
    # chunk_size = model.config.max_position_embeddings - 1
    # input_id_chunks = [input_ids[:, i:i + chunk_size] for i in range(0, input_ids.size(1), chunk_size)]
    
    # perplexities = []
    # with torch.no_grad():
    #     for chunk in input_id_chunks:
    #         outputs = model(chunk, labels=chunk, return_dict=True)
    #         loss = outputs.loss
    #         perplexity = torch.exp(loss).item()
    #         perplexities.append(perplexity)
    
    # # Average perplexities
    # corpus_perplexity = sum(perplexities) / len(perplexities)
        
    # Compute corpus-level sacreBLEU score
    bleu = BLEU()
    bleu_score = bleu.corpus_score(generated_texts, [reference_texts]).score
    
    # Compute corpus-level chrF++ score
    chrf = CHRF()
    chrf_score = chrf.corpus_score(generated_texts, [reference_texts]).score

    # Compute corpus-level CER score
    cer = load("cer")
    cer_score = cer.compute(predictions=generated_texts, references=reference_texts)

    # Compute corpus-level distinct-2 score
    distinct_2_score = distinct_n_corpus_level([text.split() for text in generated_texts], n=2)

    return bleu_score, chrf_score, cer_score, distinct_2_score

In [420]:
import json
from typing import Dict, Any

def save_results(results: Dict[str, Any], file_path: str):
    def convert_keys_to_strings(obj):
        if isinstance(obj, dict):
            return {str(k): convert_keys_to_strings(v) for k, v in obj.items()}
        elif isinstance(obj, (list, tuple)):
            return [convert_keys_to_strings(elem) for elem in obj]
        else:
            return obj

    with open(file_path, 'w') as f:
        json.dump(convert_keys_to_strings(results), f, indent=4)


In [421]:
def run_pipeline(file_path, strategies, hyperparameters):
    prompts, references = extract_prompts_and_references(file_path)

    folder_path = 'experiments/results'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Create a results dictionary to store the evaluation scores
    results = {}
    for strategy in strategies:
        results[strategy] = {}
        for hyperparameter in hyperparameters[strategy]:
            key = tuple(hyperparameter.items())  # Convert the dictionary to a tuple of tuples
            results[strategy][key] = {'bleu_scores': [], 'chrf_scores': [], 'cer_scores': [], 'distinct_2_scores': []}
    
    output_file_path = "experiments/results/transformer_output_samples.txt"
    with open(output_file_path, "w") as output_file:

        # Loop through each decoding strategy with the specified hyperparameters
        for strategy in strategies:
            for hyperparameter in hyperparameters[strategy]:
                key = tuple(hyperparameter.items())  # Convert the dictionary to a tuple of tuples
                generated_texts_list = []
                for i in range(len(prompts)):
                    prompt = prompts[i]
                    reference = references[i]
                    if not hyperparameter:  # Check if the hyperparameter list is empty
                        generated_text = generate_text(prompt, strategy, {})
                    else:
                        generated_text = generate_text(prompt, strategy, hyperparameter)
                    generated_texts_list.append(generated_text)

                    output_file.write(f"Prompt: {prompt}\n")
                    output_file.write(f"Strategy: {strategy}\n")
                    output_file.write(f"Hyperparameters: {hyperparameter}\n")
                    output_file.write(f"Generated text: {generated_text}\n")
                    output_file.write(f"Reference text: {reference}\n\n")
                
                # Evaluate all generated texts using the modified evaluate function
                bleu_score, chrf_score, cer_score, distinct_2_score = evaluate(generated_texts_list, references)

                # Save the results to the results dictionary
                results[strategy][key]['bleu_scores'].append(bleu_score)
                results[strategy][key]['chrf_scores'].append(chrf_score)
                results[strategy][key]['cer_scores'].append(cer_score)
                results[strategy][key]['distinct_2_scores'].append(distinct_2_score)

                output_file.write(f"Strategy: {strategy}\n")
                output_file.write(f"Hyperparameters: {hyperparameter}\n")
                output_file.write(f"BLEU score: {bleu_score}\n")
                output_file.write(f"chrF++ score: {chrf_score}\n")
                output_file.write(f"CER score: {cer_score}\n")
                output_file.write(f"Distinct-2 score: {distinct_2_score}\n\n")

    results_file_path = 'experiments/results/transformer_results.json'
    save_results(results, results_file_path)

    return results


In [422]:
strategies = ['argmax', 'beam_search', 'nucleus_sampling', 'top_k_sampling', 'temperature_sampling', 'entmax_sampling']
hyperparameters = {
    'argmax': [
        {'num_beams': 1}
    ],
    'beam_search': [
        {'beam_size': 5},
        {'beam_size': 10},
        {'beam_size': 15},
    ],
    'nucleus_sampling': [
        {'top_p': 0.45},
        {'top_p': 0.85},
        {'top_p': 0.90},
        {'top_p': 0.95},
    ],
    'top_k_sampling': [
        {'k': 10},
        {'k': 20},
        {'k': 40},
        {'k': 80},
    ],
    'temperature_sampling': [
        {'temperature': 0.5},
        {'temperature': 0.7},
        {'temperature': 0.9},
    ],
    'entmax_sampling': [
        {'alpha': 1.2},
        {'alpha': 1.3},
        {'alpha': 1.4},
        {'alpha': 1.5},
    ],
}

file_path = 'data/test/isolezwe.txt'

results = run_pipeline(file_path, strategies, hyperparameters)

### AWD-LSTM Language Model

In [348]:
import sys
sys.path.insert(0,'/Users/victoriapedlar/repos/isizulu-text-generation/awd_lstm')

In [349]:
import torch.nn as nn
from model import LSTMModel
from drop_connect import WeightDrop
from locked_dropout import LockedDropout
from embedding_dropout import embedded_dropout

In [350]:
# Load the model from the .pt file
model_path = "experiments/trained_models/awd_lstm/model.pt"
model= torch.load(model_path, map_location=torch.device('cpu'))

In [351]:
# Load the saved Corpus object
corpus = torch.load("experiments/trained_models/awd_lstm/corpus.data")

# Access the tokenizer (dictionary) from the loaded Corpus object
dictionary = corpus.dictionary

dictionary.unk_index = 0

In [352]:
def tokenize_text(text, dictionary):
    tokens = []
    for word in text.split():
        if word in dictionary.word2idx:
            tokens.append(dictionary.word2idx[word])
        else:
            tokens.append(dictionary.unk_index)
    return tokens

In [353]:
def sample_with_temperature(logits, temperature):
    word_weights = logits.squeeze().div(temperature).exp().cpu()
    word_idx = torch.multinomial(word_weights, 1)[0]
    return word_idx.item()

In [354]:
def beam_search(prompt_tokens, initial_hidden, k=3, temperature=1.0, max_length=round(average_tokens)):
    # Initialize beams with (tokens, hidden_state, log_prob)
    beams = [([token], initial_hidden, 0.0) for token in prompt_tokens]  # (tokens, hidden_state, log_prob)

    for _ in range(max_length):
        new_beams = []

        for tokens, hidden, log_prob in beams:
            # Create a new input tensor by taking the last token
            token_input = torch.tensor([[tokens[-1]]], dtype=torch.long)
            
            logits, new_hidden = model(token_input, hidden)
            last_token_logits = logits.squeeze()

            top_k_logits, top_k_indices = torch.topk(last_token_logits, k)
            top_k_probs = torch.softmax(top_k_logits / temperature, dim=-1)

            for prob, idx in zip(top_k_probs.squeeze(), top_k_indices.squeeze()):
                new_tokens = tokens + [idx.item()]
                new_prob = log_prob + prob.item()
                new_beams.append((new_tokens, new_hidden, new_prob))

        beams = sorted(new_beams, key=lambda x: x[2], reverse=True)[:k]

    best_beam = torch.tensor(beams[0][0], dtype=torch.long).unsqueeze(0)
    return best_beam

In [355]:
def entmax_sampling_step(logits, alpha=1.5):
    probabilities = entmax15(logits, dim=-1)
    word_idx = torch.multinomial(probabilities, 1)[0]
    return word_idx.item()

In [360]:
import numpy as np
from entmax import entmax15
from torch.autograd import Variable

def generate_text_awd_lstm(prompt, strategy, hyperparameters, max_length=max_length):

    model.eval()
    with torch.no_grad():
        input_prompt_tokens = tokenize_text(prompt, dictionary)
        
        # Initialize hidden state for a single token
        hidden = model.init_hidden(1)

        # Process input_prompt_tokens one by one to update the hidden state
        for token in input_prompt_tokens:
            input_token = torch.tensor([[token]], dtype=torch.long)
            _, hidden = model(input_token, hidden)

        generated_tokens = []
        last_token = input_prompt_tokens[-1] if input_prompt_tokens else None

    for _ in range(max_length):
        if last_token is not None:
            input_token = torch.tensor([[last_token]], dtype=torch.long)
            logits, hidden = model(input_token, hidden)
            last_token_logits = logits[:, -1, :]
        else:
            # Placeholder for cases when input_prompt_tokens is empty
            last_token_logits = torch.zeros(1, len(dictionary.word2idx))

        if strategy == 'beam_search':
            k = hyperparameters.get('k', 3)
            temperature = hyperparameters.get('temperature', 1.0)
            best_beam = beam_search(input_prompt_tokens, hidden, k=k, temperature=temperature)
            generated_tokens = best_beam.squeeze().tolist()[len(input_prompt_tokens):]
            break
        else:
            # The code for other strategies should be indented to be inside the for-loop
            if strategy == 'argmax':
                dim = hyperparameters.get('dim', -1)
                token_index = torch.argmax(last_token_logits, dim=dim).item()
            elif strategy == 'nucleus_sampling':
                top_p = hyperparameters.get('top_p', 0.9)
                sorted_logits, sorted_indices = torch.sort(last_token_logits, descending=True)
                cumulative_probs = torch.cumsum(torch.softmax(sorted_logits, dim=-1), dim=-1)
                indices_to_remove = cumulative_probs > top_p
                indices_to_remove[..., 1:] = indices_to_remove[..., :-1].clone()
                indices_to_remove[..., 0] = 0
                sorted_logits[indices_to_remove] = float('-inf')
                probabilities = torch.softmax(sorted_logits, dim=-1)
                token_index = np.random.choice(len(dictionary.word2idx), p=probabilities.squeeze().cpu().detach().numpy())
            elif strategy == 'top_k_sampling':
                k = hyperparameters.get('k', 10)
                top_k_logits, _ = torch.topk(last_token_logits, k)
                min_top_k_logits = torch.min(top_k_logits, dim=-1).values.unsqueeze(-1)
                last_token_logits[last_token_logits < min_top_k_logits] = float('-inf')
                probabilities = torch.softmax(last_token_logits, dim=-1)
                token_index = np.random.choice(len(dictionary.word2idx), p=probabilities.squeeze().cpu().detach().numpy())
            elif strategy == 'temperature_sampling':
                temperature = hyperparameters.get('temperature', 1.0)
                token_index = sample_with_temperature(last_token_logits, temperature)
            elif strategy == 'entmax_sampling':
                alpha = hyperparameters.get('alpha', 1.5)
                token_index = entmax_sampling_step(last_token_logits, alpha)
            else:
                raise ValueError(f"Invalid strategy: {strategy}")

            generated_tokens.append(token_index)
            last_token = token_index
            
    generated_text = [dictionary.idx2word[token_idx] for token_idx in generated_tokens]
    generated_text = [dictionary.idx2word[token_idx].replace('Ġ', '').replace('Ċ', ' ') for token_idx in generated_tokens]
    generated_text = " ".join(generated_text)
    generated_text = generated_text.replace('\n', '')

    return generated_text

In [361]:
def evaluate_awd_lstm(generated_texts, reference_texts):

    # # Ensure each generated text is on a single line
    # cleaned_generated_texts = [' '.join(text.strip().split()) for text in generated_texts]
    
    # # Compute perplexity for each generated text
    # perplexities = []
    # for text in cleaned_generated_texts:
    #     input_ids = torch.tensor([tokenize_text(text, dictionary)], dtype=torch.long)
    #     hidden = model.init_hidden(input_ids.size(1))
    #     with torch.no_grad():
    #         outputs, _ = model(input_ids, hidden)
    #         logits = outputs[:, :-1, :]  # Remove the last token (there is no next token to predict)
    #         labels = input_ids[:, 1:]  # Remove the first token (there is no previous token)
    #         loss = torch.nn.functional.cross_entropy(logits.reshape(-1, logits.size(-1)), labels.reshape(-1))
    #         perplexity = torch.exp(loss).item()
    #         perplexities.append(perplexity)

    # # Average perplexities
    # corpus_perplexity = sum(perplexities) / len(perplexities)
        
    # Compute corpus-level sacreBLEU score
    bleu = BLEU()
    bleu_score = bleu.corpus_score(generated_texts, [reference_texts]).score
    
    # Compute corpus-level chrF++ score
    chrf = CHRF()
    chrf_score = chrf.corpus_score(generated_texts, [reference_texts]).score

    # Compute corpus-level CER score
    cer = load("cer")
    cer_score = cer.compute(predictions=generated_texts, references=reference_texts)

    # Compute corpus-level distinct-N score
    distinct_2_score = distinct_n_corpus_level([text.split() for text in generated_texts], n=2)

    return bleu_score, chrf_score, cer_score, distinct_2_score

In [362]:
import os

def run_pipeline(file_path, strategies, hyperparameters):
    
    prompts, references = extract_prompts_and_references(file_path)

    folder_path = 'experiments/results'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Create a results dictionary to store the evaluation scores
    results = {}
    for strategy in strategies:
        results[strategy] = {}
        for hyperparameter in hyperparameters[strategy]:
            key = tuple(hyperparameter.items())  # Convert the dictionary to a tuple of tuples
            results[strategy][key] = {'bleu_scores': [], 'chrf_scores': [], 'cer_scores': [], 'distinct_2_scores': []}
    
    output_file_path = "experiments/results/awd_lstm_output_samples.txt"
    with open(output_file_path, "w") as output_file:

        # Loop through each decoding strategy with the specified hyperparameters
        for strategy in strategies:
            for hyperparameter in hyperparameters[strategy]:
                key = tuple(hyperparameter.items())  # Convert the dictionary to a tuple of tuples
                generated_texts_list = []
                for i in range(len(prompts)):
                    prompt = prompts[i]
                    reference = references[i]
                    if not hyperparameter:  # Check if the hyperparameter list is empty
                        generated_text = generate_text_awd_lstm(prompt, strategy, {})
                    else:
                        generated_text = generate_text_awd_lstm(prompt, strategy, hyperparameter)
                    generated_texts_list.append(generated_text)

                    output_file.write(f"Prompt: {prompt}\n")
                    output_file.write(f"Strategy: {strategy}\n")
                    output_file.write(f"Hyperparameters: {hyperparameter}\n")
                    output_file.write(f"Generated text: {generated_text}\n")
                    output_file.write(f"Reference text: {reference}\n\n")
                
                # Evaluate all generated texts using the modified evaluate function
                bleu_score, chrf_score, cer_score, distinct_2_score = evaluate_awd_lstm(generated_texts_list, references)

                # Save the results to the results dictionary
                results[strategy][key]['bleu_scores'].append(bleu_score)
                results[strategy][key]['chrf_scores'].append(chrf_score)
                results[strategy][key]['cer_scores'].append(cer_score)
                results[strategy][key]['distinct_2_scores'].append(distinct_2_score)

                output_file.write(f"Strategy: {strategy}\n")
                output_file.write(f"Hyperparameters: {hyperparameter}\n")
                output_file.write(f"BLEU score: {bleu_score}\n")
                output_file.write(f"chrF++ score: {chrf_score}\n")
                output_file.write(f"CER score: {cer_score}\n")
                output_file.write(f"Distinct-2 score: {distinct_2_score}\n\n")

    results_file_path = 'experiments/results/awd_lstm_results.json'
    save_results(results, results_file_path)

    return results

In [363]:
strategies = ['argmax', 'beam_search', 'nucleus_sampling', 'top_k_sampling', 'temperature_sampling', 'entmax_sampling']
hyperparameters = {
    'argmax': [
        {'dim': -1}
    ],
    'beam_search': [
        {'beam_size': 5},
        {'beam_size': 10},
        {'beam_size': 15},
    ],
    'nucleus_sampling': [
        {'top_p': 0.45},
        {'top_p': 0.85},
        {'top_p': 0.90},
        {'top_p': 0.95},
    ],
    'top_k_sampling': [
        {'k': 10},
        {'k': 20},
        {'k': 40},
        {'k': 80},
    ],
    'temperature_sampling': [
        {'temperature': 0.5},
        {'temperature': 0.7},
        {'temperature': 0.9},
    ],
    'entmax_sampling': [
        {'alpha': 1.2},
        {'alpha': 1.3},
        {'alpha': 1.4},
        {'alpha': 1.5},
    ],
}

file_path = 'data/test/isolezwe.txt'

results = run_pipeline(file_path, strategies, hyperparameters)

### Sparse Language Model

In [364]:
import sys
sys.path.insert(0,'/Users/victoriapedlar/repos/isizulu-text-generation/sparse_text_generation/language_modeling/pytorch_transformers')
from transformers import (
    GPT2LMHeadModel,
    GPT2TokenizerFast
)

In [365]:
tokenizer = GPT2TokenizerFast.from_pretrained('experiments/trained_models/sparse_model/tokenizers')
# Load the model
model = GPT2LMHeadModel.from_pretrained("experiments/trained_models/sparse_model")
# Set the tokenizer attribute of the model
model.tokenizer = tokenizer

In [366]:
def run_pipeline(file_path, strategies, hyperparameters):
    prompts, references = extract_prompts_and_references(file_path)

    folder_path = 'experiments/results'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Create a results dictionary to store the evaluation scores
    results = {}
    for strategy in strategies:
        results[strategy] = {}
        for hyperparameter in hyperparameters[strategy]:
            key = tuple(hyperparameter.items())  # Convert the dictionary to a tuple of tuples
            results[strategy][key] = {'bleu_scores': [], 'chrf_scores': [], 'cer_scores': [], 'distinct_2_scores': []}
    
    output_file_path = "experiments/results/sparse_output_samples.txt"
    with open(output_file_path, "w") as output_file:

        # Loop through each decoding strategy with the specified hyperparameters
        for strategy in strategies:
            for hyperparameter in hyperparameters[strategy]:
                key = tuple(hyperparameter.items())  # Convert the dictionary to a tuple of tuples
                generated_texts_list = []
                for i in range(len(prompts)):
                    prompt = prompts[i]
                    reference = references[i]
                    if not hyperparameter:  # Check if the hyperparameter list is empty
                        generated_text = generate_text(prompt, strategy, {})
                    else:
                        generated_text = generate_text(prompt, strategy, hyperparameter)
                    generated_texts_list.append(generated_text)

                    output_file.write(f"Prompt: {prompt}\n")
                    output_file.write(f"Strategy: {strategy}\n")
                    output_file.write(f"Hyperparameters: {hyperparameter}\n")
                    output_file.write(f"Generated text: {generated_text}\n")
                    output_file.write(f"Reference text: {reference}\n\n")
                
                # Evaluate all generated texts using the modified evaluate function
                bleu_score, chrf_score, cer_score, distinct_2_score = evaluate(generated_texts_list, references)

                # Save the results to the results dictionary
                results[strategy][key]['bleu_scores'].append(bleu_score)
                results[strategy][key]['chrf_scores'].append(chrf_score)
                results[strategy][key]['cer_scores'].append(cer_score)
                results[strategy][key]['distinct_1_scores'].append(distinct_2_score)

                output_file.write(f"Strategy: {strategy}\n")
                output_file.write(f"Hyperparameters: {hyperparameter}\n")
                output_file.write(f"BLEU score: {bleu_score}\n")
                output_file.write(f"chrF++ score: {chrf_score}\n")
                output_file.write(f"CER score: {cer_score}\n")
                output_file.write(f"Distinct-2 score: {distinct_2_score}\n\n")

    results_file_path = 'experiments/results/sparse_results.json'
    save_results(results, results_file_path)

    return results


In [367]:
strategies = ['argmax', 'beam_search', 'nucleus_sampling', 'top_k_sampling', 'temperature_sampling', 'entmax_sampling']
hyperparameters = {
    'argmax': [
        {'num_beams': 1}
    ],
    'beam_search': [
        {'beam_size': 5},
        {'beam_size': 10},
        {'beam_size': 15},
    ],
    'nucleus_sampling': [
        {'top_p': 0.45},
        {'top_p': 0.85},
        {'top_p': 0.90},
        {'top_p': 0.95},
    ],
    'top_k_sampling': [
        {'k': 10},
        {'k': 20},
        {'k': 40},
        {'k': 80},
    ],
    'temperature_sampling': [
        {'temperature': 0.5},
        {'temperature': 0.7},
        {'temperature': 0.9},
    ],
    'entmax_sampling': [
        {'alpha': 1.2},
        {'alpha': 1.3},
        {'alpha': 1.4},
        {'alpha': 1.5},
    ],
}

file_path = 'data/test/isolezwe.txt'

results = run_pipeline(file_path, strategies, hyperparameters)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


KeyboardInterrupt: 